### Simple Wrappers

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents.Wrappers import *

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'

cyborg = CybORG(path,'sim')

Our demonstrations have mostly been with native CybORG, but we will now demonstrate how AI can be assisted by the use of Wrappers.

Our first Wrapper is EnumActionWrapper, which calculates all the possible actions and returns the action space as the number of such actions. This gives a relatively large number of actions for both Blue and Red team.

In [2]:
env = EnumActionWrapper(cyborg)

results = env.reset(agent='Blue')
action_space = results.action_space
print('Blue action space:',action_space)

results = env.reset(agent='Red')
action_space = results.action_space
print('Red action space:', action_space)

Blue action space: 158
Red action space: 888


We can do a similar thing for the observation space. The FlatFixedWrapper parses the internal state of CybORG and turns it into a list of floats, which can easily be converted into a vector. Unfortunately, this vector is extremely long, with length over 11,000!

In [3]:
env = FixedFlatWrapper(CybORG(path, 'sim'))

results = env.reset()
obs = results.observation
print(type(obs))
print(len(obs))

<class 'list'>
11293


The OpenAIGymWrapper converts the output of FlatFixedWrapper to a numpy array as well as conforming to other parts of the OpenAI Gym API. It requires FlatFixedWrapper and EnumActionWrapper in order to function and should always be the outermost of the provided wrappers. You must also specify an agent parameter and explitly specify the environment parameter.

In [4]:
wrappers = FixedFlatWrapper(EnumActionWrapper(cyborg))
env = OpenAIGymWrapper(env=wrappers,agent_name='Blue')

obs = env.reset()
print('Observation:',obs)
print(73*'-')
print('Action_Space:',env.action_space)
print(73*'-')
print('Observation Space:',env.observation_space)

Observation: [ 0.66666667  0.          1.         ... -1.         -1.
 -1.        ]
-------------------------------------------------------------------------
Action_Space: Discrete(158)
-------------------------------------------------------------------------
Observation Space: Box([-1. -1. -1. ... -1. -1. -1.], [1. 1. 1. ... 1. 1. 1.], (11293,), float32)


### Table Wrappers

The 'Table Wrappers' attempt to use basic logic to infer a human-friendly picture of the state by keeping track of past observations. This allows for a greatly simplified state space and much greater human readibility. However, it mercilessly exploits the current limitations of Scenario 1b and thus would have limited use on real-world cyber problems.

The first wrapper is the TrueTableWrapper, which modifies the get_agent_state method to return the true state in the form of the table.

In [5]:
env = TrueTableWrapper(cyborg)

env.reset()

true_table = env.get_agent_state('True')
print(true_table)

+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
| 10.0.209.112/28 | 10.0.209.114 |   Defender  | False |  False  |    None    |
| 10.0.209.112/28 | 10.0.209.120 | Enterprise0 | False |  False  |    None    |
| 10.0.209.112/28 | 10.0.209.117 | Enterprise1 | False |  False  |    None    |
| 10.0.209.112/28 | 10.0.209.118 | Enterprise2 | False |  False  |    None    |
|  10.0.210.96/28 | 10.0.210.102 |   Op_Host0  | False |  False  |    None    |
|  10.0.210.96/28 | 10.0.210.100 |   Op_Host1  | False |  False  |    None    |
|  10.0.210.96/28 | 10.0.210.103 |   Op_Host2  | False |  False  |    None    |
|  10.0.210.96/28 | 10.0.210.98  |  Op_Server0 | False |  False  |    None    |
|  10.0.146.32/28 | 10.0.146.37  |    User0    |  True |  False  | Privileged |
|  10.0.146.32/28 | 10.0.146.46  |    Us

The BlueTableWrapper provides similar functionality for the blue observation.

In [6]:
from CybORG.Agents import B_lineAgent
from CybORG.Shared.Actions import Sleep
cyborg = CybORG(path,'sim', agents={'Red':B_lineAgent})

env = BlueTableWrapper(cyborg)

results = env.reset(agent='Blue')

for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)

+-----------------+--------------+-------------+----------+-------------+
|      Subnet     |  IP Address  |   Hostname  | Activity | Compromised |
+-----------------+--------------+-------------+----------+-------------+
|  10.0.11.160/28 | 10.0.11.164  |   Defender  |   None   |      No     |
|  10.0.11.160/28 | 10.0.11.165  | Enterprise0 |   None   |      No     |
|  10.0.11.160/28 | 10.0.11.172  | Enterprise1 |   None   |      No     |
|  10.0.11.160/28 | 10.0.11.163  | Enterprise2 |   None   |      No     |
| 10.0.199.208/28 | 10.0.199.220 |   Op_Host0  |   None   |      No     |
| 10.0.199.208/28 | 10.0.199.210 |   Op_Host1  |   None   |      No     |
| 10.0.199.208/28 | 10.0.199.213 |   Op_Host2  |   None   |      No     |
| 10.0.199.208/28 | 10.0.199.219 |  Op_Server0 |   None   |      No     |
|  10.0.11.48/28  |  10.0.11.59  |    User0    |   None   |      No     |
|  10.0.11.48/28  |  10.0.11.58  |    User1    |   None   |      No     |
|  10.0.11.48/28  |  10.0.11.60  |    

The table can also be converted into a vector. This is done by setting the output_mode parameter to 'vector'.

In [7]:
env = BlueTableWrapper(cyborg,output_mode='vector')

env.reset(agent='Blue')
for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)
    print(76*'-')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------


The RedTableWrapper is the Red Team version of the BlueTableWrapper.

In [8]:
env = RedTableWrapper(cyborg)

results = env.reset(agent='Red')
print(results.observation)

for i in range(3):
    results = env.step(agent='Red')
    red_obs = results.observation
    print(red_obs)

+-----------------+--------------+----------+---------+------------+
|      Subnet     |  IP Address  | Hostname | Scanned |   Access   |
+-----------------+--------------+----------+---------+------------+
| 10.0.219.192/28 | 10.0.219.194 |  User0   |  False  | Privileged |
+-----------------+--------------+----------+---------+------------+
+-----------------+--------------+-----------------+---------+------------+
|      Subnet     |  IP Address  |     Hostname    | Scanned |   Access   |
+-----------------+--------------+-----------------+---------+------------+
| 10.0.219.192/28 | 10.0.219.193 | UNKNOWN_HOST: 2 |  False  |    None    |
| 10.0.219.192/28 | 10.0.219.194 |      User0      |  False  | Privileged |
| 10.0.219.192/28 | 10.0.219.196 | UNKNOWN_HOST: 0 |  False  |    None    |
| 10.0.219.192/28 | 10.0.219.198 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.219.192/28 | 10.0.219.199 | UNKNOWN_HOST: 1 |  False  |    None    |
+-----------------+--------------+-------------

### Challenge Wrapper

The challenge wrapper is three wrappers nested together: BlueTableWrapper, EnumActionWrapper and OpenAIGymWrapper.

In [9]:
env = ChallengeWrapper(env=cyborg,agent_name='Red')

obs = env.reset()

for i in range(1):
    obs, reward, done, info = env.step()
    print(obs)
    print(76*'-')
    print(reward)
    print(76*'-')
    print(done)
    print(76*'-')
    pprint(info)

[ 1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
----------------------------------------------------------------------------
0.0
----------------------------------------------------------------------------
False
----------------------------------------------------------------------------
{'action': <CybORG.Shared.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems object at 0x7fbdfc1879a0>,
 'action_name': None,
 'action_space': 888,
 'done': False,
 'error': None,
 'error_msg': None,
 'info': None,
 'next_observation': None,
 'observation': array([ 1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1]),
 'parameter_mask': None,
 'reward': 0.0,
 'selection_masks': None}
